# Using PowerSystems to calculate network matrices

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSystems.jl supports the calculation of several different matrix representations of
power system networks. This example demonstrates how to use PowerSystems.jl to calculate:
 - Y bus
 - Power transfer distribution factor (PTDF)
 - Line outage distribution  factor (LODF)

### Dependencies
Let's use a dataset from the [tabular data parsing example](../../notebook/PowerSystems_examples/parse_matpower.ipynb)

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath,"test", "PowerSystems_examples", "parse_matpower.jl"))

Project SIIPExamples v0.0.1
    Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [9961bab8] Cbc v0.6.7
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.20.0
  [2cd47ed4] InfrastructureSystems v0.5.13
  [b6b21f68] Ipopt v0.6.1
  [2535ab7d] JSON2 v0.3.1
  [4076af6c] JuMP v0.21.1
  [98b081ad] Literate v2.3.0
  [47be7bcc] ORCA v0.3.1
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v0.29.8
  [5f7eddb3] PowerGraphics v0.2.0
  [e690365d] PowerSimulations v0.3.1
  [bcd98974] PowerSystems v0.12.0
  [9e3dc215] TimeSeries v0.16.0
  [0f1e0344] WebIO v0.8.13
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [44cfe95a] Pkg 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 
┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Zvssm/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Zvssm/src/parsers/pm_io/ma

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
3,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
4,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
6,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
7,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
8,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


### Ybus

In [2]:
ybus = Ybus(sys)

PowerNetworkMatrix
:
  22.2507-222.484im  -3.52348+35.2348im  …  -15.4703+154.703im
 -3.52348+35.2348im   12.6911-126.898im          0.0+0.0im    
      0.0+0.0im      -9.16758+91.6758im          0.0+0.0im    
  -3.2569+32.569im        0.0+0.0im         -3.33367+33.3367im
 -15.4703+154.703im       0.0+0.0im           18.804-188.021im

### PTDF

In [3]:
ptdf = PTDF(sys)

PowerNetworkMatrix
:
 -0.000521593  0.0  -0.000429122  -0.00140973  -0.00175107
 -0.000521593  0.0  -0.000429122  -0.00140973  -0.00175107
 -0.00104319   0.0  -0.000858244  -0.00281945  -1.0035    
 -0.00104319   0.0  -0.000858244  -1.00282     -1.0035    
  0.543423     0.0   0.447082      0.544388     0.544758  
 -0.45762      0.0  -0.553776     -0.458432    -0.458744  
  0.45762      0.0  -0.446224      0.458432     0.458744  

### LODF

In [4]:
lodf = LODF(sys)

PowerNetworkMatrix
:
 -1.0         -0.00174801   0.000341347  …   0.000961673  -0.000961673
 -0.00174801  -1.0          0.000341347      0.000961673  -0.000961673
 -1.00175     -1.00175     -1.0              0.00192335   -0.00192335 
 -1.00175     -1.00175      0.000682694      0.00192335   -0.00192335 
  0.543806     0.543806    -0.000370605     -1.00192       1.00192    
 -0.457942    -0.457942     0.000312089  …  -1.0           1.0        
  0.457942     0.457942    -0.000312089      1.0          -1.0        

### Indexing
Note that the axes of these matrices that correspond to buses are indexed by bus number
(::Int64) while the branch axes are indexed by branch name (::String). You can access
specific elements of the matrices as follows:

In [5]:
ptdf["5",3]

-0.0017510735223031867

Additionally, PowerSystems provides accessors to the network matrices that take `Componets`
as arguments so that you can pass references to the components themselves rather than the
name or number. For example:

In [6]:
buses = collect(get_components(Bus,sys))
ybus[buses[1],buses[2]]

-3.2569046378322044 + 32.56904637832204im

If you would instead like to index by bus name, something like the following should work:

In [7]:
busname2num = get_components(Bus,sys) |> (c -> Dict(zip(get_name.(c), get_number.(c))))
ptdf["5", busname2num["bus3"]]

-0.0017510735223031867

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*